In [1]:
import tensorflow as tf
# #指定使用那块GUP训练
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "1"
config = tf.ConfigProto()
# 设置最大占有GPU不超过显存的70%
config.gpu_options.per_process_gpu_memory_fraction = 0.7 
# # 重点：设置动态分配GPU
config.gpu_options.allow_growth = True
# 创建session时
tf.Session(config=config) 

In [ ]:
import numpy as np
from keras.datasets import cifar10
from torch.utils.data import Dataset
import torch.utils.data as Data
class MyDataset(Dataset):
    def __init__(self, imgs, transform=None):
        # super().__init__()
        self.imgs = imgs
        self.transform = transform

    def __len__(self):
        return len(self.imgs)

    def __getitem__(self, index):
        img = self.imgs[index]
        if self.transform is not None:
            img = self.transform(img)
        else:
            img = torch.from_numpy(img)
        img=img.reshape([3,32,32])
        return img
import torch
from torch.autograd import Variable
import model
import torch.nn.functional as F
model = model.cifar10(128)
model.load_state_dict(torch.load('./log/default/best-85.pth'))
model.cuda()
def EuclideanDistances(A, B):
    BT = B.transpose()
    # vecProd = A * BT
    vecProd = np.dot(A, BT)
    # print(vecProd)
    SqA = A ** 2
    # print(SqA)
    sumSqA = np.matrix(np.sum(SqA, axis=1))
    sumSqAEx = np.tile(sumSqA.transpose(), (1, vecProd.shape[1]))
    # print(sumSqAEx)

    SqB = B ** 2
    sumSqB = np.sum(SqB, axis=1)
    sumSqBEx = np.tile(sumSqB, (vecProd.shape[0], 1))
    SqED = sumSqBEx + sumSqAEx - 2 * vecProd
    SqED[SqED < 0] = 0.0
    ED = np.sqrt(SqED)
    return np.divide(ED.sum(), ED.shape[0] * ED.shape[1])


def cal_distance_image_real(images, labels):
    x_dataset = MyDataset(images)
    # print(x_dataset[0].shape)
    x_real_loader = Data.DataLoader(dataset=x_dataset, batch_size=200, shuffle=True)
    y_logits = []
    for i, data in enumerate(x_real_loader):
        # indx_target = target.clone()
        data = data.cuda()
        data = Variable(data, volatile=True)
        output = model(data)
        pred = F.softmax(output).cpu().detach().numpy()
        y_logits += [i for i in pred]
    dict = {}
    all_dis = []
    for i in range(10):
        dict[i] = []
    for i in range(len(labels)):
        dict[labels[i]].append(y_logits[i])
    for i in range(10):
        dict[i] = np.array(dict[i])
        if len(dict[i]):
            dis = EuclideanDistances(dict[i], dict[i])  # 生成图片的紧度
        else:
            dis = -1
        all_dis.append(dis)
    return np.array(all_dis)


def cal_distance_image_fake(images):
    x_dataset = MyDataset(images)
    # print(x_dataset[0].shape)
    x_real_loader = Data.DataLoader(dataset=x_dataset, batch_size=200, shuffle=True)
    y_logits = []
    labels=[]
    for i, data in enumerate(x_real_loader):
        # indx_target = target.clone()
        data = data.cuda()
        data = Variable(data, volatile=True)
        output = model(data)
        pred = output.data.max(1)[1]
        labels += [i for i in pred.cpu().numpy()]
        pred = F.softmax(output).cpu().detach().numpy()
        y_logits += [i for i in pred]

    dict = {}
    all_dis = []
    for i in range(10):
        dict[i] = []
    for i in range(len(labels)):
        dict[labels[i]].append(y_logits[i])
    for i in range(10):
        dict[i] = np.array(dict[i])
        if len(dict[i]):
            dis = EuclideanDistances(dict[i], dict[i])  # 生成图片的紧度
        else:
            dis = -1
        all_dis.append(dis)
    return np.array(all_dis)

import os
if not os.path.isdir('saved_models_{}'.format('bgan')):
    os.mkdir('saved_models_{}'.format('bgan'))
f = open('saved_models_{}/log_collapse1.txt'.format('bgan'), mode='w')
import torch.utils.data as Data
import cv2

# Large amount of credit goes to:
# https://github.com/keras-team/keras-contrib/blob/master/examples/improved_wgan.py
# which I've used as a reference for this implementation
from keras.datasets import cifar10
from keras.layers.merge import _Merge
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import RMSprop
from functools import partial
from PIL import Image
import keras.backend as K

import matplotlib.pyplot as plt

import sys

import numpy as np
import os
class RandomWeightedAverage(_Merge):
    """Provides a (random) weighted average between real and generated image samples"""
    def _merge_function(self, inputs):
        alpha = K.random_uniform((32, 1, 1, 1))
        return (alpha * inputs[0]) + ((1 - alpha) * inputs[1])

class WGANGP():
    def __init__(self):
        self.img_rows = 32
        self.img_cols = 32
        self.channels = 3
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100
        self.x = []
        self.y = np.zeros((31, 1), dtype=np.int)
        self.y = list(self.y)
        for i in range(31):
            self.y[i] = []

        # Following parameter and optimizer set as recommended in paper
        self.n_critic = 5
        optimizer = RMSprop(lr=0.00005)

        # Build the generator and critic
        self.generator = self.build_generator()
        self.critic = self.build_critic()

        #-------------------------------
        # Construct Computational Graph
        #       for the Critic
        #-------------------------------

        # Freeze generator's layers while training critic
        self.generator.trainable = False

        # Image input (real sample)
        real_img = Input(shape=self.img_shape)

        # Noise input
        z_disc = Input(shape=(self.latent_dim,))
        # Generate image based of noise (fake sample)
        fake_img = self.generator(z_disc)

        # Discriminator determines validity of the real and fake images
        fake = self.critic(fake_img)
        valid = self.critic(real_img)

        # Construct weighted average between real and fake images
        interpolated_img = RandomWeightedAverage()([real_img, fake_img])
        # Determine validity of weighted sample
        validity_interpolated = self.critic(interpolated_img)

        # Use Python partial to provide loss function with additional
        # 'averaged_samples' argument
        partial_gp_loss = partial(self.gradient_penalty_loss,
                          averaged_samples=interpolated_img)
        partial_gp_loss.__name__ = 'gradient_penalty' # Keras requires function names

        self.critic_model = Model(inputs=[real_img, z_disc],
                            outputs=[valid, fake, validity_interpolated])
        self.critic_model.compile(loss=[self.wasserstein_loss,
                                              self.wasserstein_loss,
                                              partial_gp_loss],
                                        optimizer=optimizer,
                                        loss_weights=[1, 1, 10])
        #-------------------------------
        # Construct Computational Graph
        #         for Generator
        #-------------------------------

        # For the generator we freeze the critic's layers
        self.critic.trainable = False
        self.generator.trainable = True

        # Sampled noise for input to generator
        z_gen = Input(shape=(100,))
        # Generate images based of noise
        img = self.generator(z_gen)
        # Discriminator determines validity
        valid = self.critic(img)
        # Defines generator model
        self.generator_model = Model(z_gen, valid)
        self.generator_model.compile(loss=self.wasserstein_loss, optimizer=optimizer)


    def gradient_penalty_loss(self, y_true, y_pred, averaged_samples):
        """
        Computes gradient penalty based on prediction and weighted real / fake samples
        """
        gradients = K.gradients(y_pred, averaged_samples)[0]
        # compute the euclidean norm by squaring ...
        gradients_sqr = K.square(gradients)
        #   ... summing over the rows ...
        gradients_sqr_sum = K.sum(gradients_sqr,
                                  axis=np.arange(1, len(gradients_sqr.shape)))
        #   ... and sqrt
        gradient_l2_norm = K.sqrt(gradients_sqr_sum)
        # compute lambda * (1 - ||grad||)^2 still for each single sample
        gradient_penalty = K.square(1 - gradient_l2_norm)
        # return the mean as loss over all the batch samples
        return K.mean(gradient_penalty)


    def wasserstein_loss(self, y_true, y_pred):
        return K.mean(y_true * y_pred)

    def build_generator(self):

        model = Sequential()

        model.add(Dense(128 * 8 * 8, activation="relu", input_dim=self.latent_dim))
        model.add(Reshape((8, 8, 128)))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=4, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=4, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(Conv2D(self.channels, kernel_size=4, padding="same"))
        model.add(Activation("tanh"))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_critic(self):

        model = Sequential()

        model.add(Conv2D(16, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(32, kernel_size=3, strides=2, padding="same"))
        # model.add(ZeroPadding2D(padding=((0,1),(0,1))))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(128, kernel_size=3, strides=1, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(1))

        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size, sample_interval=50):

        # Load the dataset
        (X_train, _), (X_test, y_test) = cifar10.load_data()

        # Rescale -1 to 1
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_test = (X_test.astype(np.float32) - 127.5) / 127.5
        # X_train = np.expand_dims(X_train, axis=3)

        # Adversarial ground truths
        valid = -np.ones((batch_size, 1))
        fake =  np.ones((batch_size, 1))
        dummy = np.zeros((batch_size, 1)) # Dummy gt for gradient penalty
        nb_batches = int(X_train.shape[0] / batch_size)
        global_step = 0
        steps = []
        values = []

        for epoch in range(epochs):

            for index in range(nb_batches):
                for _ in range(self.n_critic):
                    global_step += 1
                    imgs = X_train[index * batch_size:(index + 1) * batch_size]
                    # Sample generator input
                    noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
                    # Train the critic
                    d_loss = self.critic_model.train_on_batch([imgs, noise],
                                                              [valid, fake, dummy])

                    # ---------------------
                    #  Train Generator
                    # ---------------------

                g_loss = self.generator_model.train_on_batch(noise, valid)

                # Plot the progress
                print("epoch%d step%d [D loss: %f] [G loss: %f]" % (epoch, global_step, d_loss[0], g_loss))
                sample_num=5000

                if global_step % sample_interval == 0:
                    self.mode_drop(X_test,y_test,sample_num, global_step)


    def mode_drop(self, x_test,y_test,sample_num, global_step):
        r, c = 10, 1000
        noise = np.random.normal(0, 1, (r * c, 100))
        # sampled_labels = np.array([num for _ in range(r) for num in range(c)])
        gen_imgs = self.generator.predict([noise])
        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5
        labels = np.squeeze(y_test[:sample_num])
        labels = np.squeeze(labels)
        dis_real = cal_distance_image_real(x_test[:sample_num], labels)
        dis_fake = cal_distance_image_fake(gen_imgs)
        dis_cha = dis_real - dis_fake
        print('##############')
        # print(dis_real)
        # print(dis_fake)
        print(dis_cha)
        print('##########')
        f.writelines('\n')
        f.writelines('step:' + str(global_step))
        f.writelines('\n')
        f.writelines('紧度')
        f.writelines('\n')
        f.writelines(' %.8f ' % (i) for i in dis_cha)
        f.writelines('\n')
if __name__ == '__main__':
    wgan = WGANGP()
    wgan.train(epochs=20, batch_size=32, sample_interval=200)


Using TensorFlow backend.
Sequential(
  (0): Conv2d(3, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  (2): ReLU()
  (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  (5): ReLU()
  (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (7): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  (9): ReLU()
  (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  (12): ReLU()
  (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (14): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 8192)              827392    
_________________________________________________________________
reshape_1 (Reshape)          (None, 8, 8, 128)         0         
_________________________________________________________________
up_sampling2d_1 (UpSampling2 (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 128)       262272    
_________________________________________________________________
batch_normalization_1 (Batch (None, 16, 16, 128)       512       
_________________________________________________________________
activation_1 (Activation)    (None, 16, 16, 128)       0         
_________________________________________________________________
up_sampling2d_2 (UpSampling2 (None, 32, 32, 128)       0         
__________

/home/imi432_006/anaconda3/envs/tf/lib/python3.5/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


epoch0 step5 [D loss: 37.600552] [G loss: 0.348166]
epoch0 step10 [D loss: 26.498463] [G loss: 0.558376]
epoch0 step15 [D loss: 25.635786] [G loss: 0.382048]
epoch0 step20 [D loss: 17.209267] [G loss: 0.048804]
epoch0 step25 [D loss: 16.879993] [G loss: 0.459808]
epoch0 step30 [D loss: 15.406905] [G loss: 0.572439]
epoch0 step35 [D loss: 11.196810] [G loss: 0.226548]
epoch0 step40 [D loss: 12.312521] [G loss: 0.434739]
epoch0 step45 [D loss: 10.865988] [G loss: 0.591425]
epoch0 step50 [D loss: 9.652449] [G loss: 0.632120]
epoch0 step55 [D loss: 9.086752] [G loss: 0.744493]
epoch0 step60 [D loss: 7.829864] [G loss: 0.639068]
epoch0 step65 [D loss: 5.584886] [G loss: 0.583557]
epoch0 step70 [D loss: 3.781470] [G loss: 0.707782]
epoch0 step75 [D loss: 4.871698] [G loss: 0.532278]
epoch0 step80 [D loss: 2.293085] [G loss: 0.721507]
epoch0 step85 [D loss: 3.726917] [G loss: 0.818822]
epoch0 step90 [D loss: 2.854299] [G loss: 0.824548]
epoch0 step95 [D loss: 2.388498] [G loss: 0.678462]
epoc

/home/imi432_006/anaconda3/envs/tf/lib/python3.5/site-packages/ipykernel_launcher.py:57: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/home/imi432_006/anaconda3/envs/tf/lib/python3.5/site-packages/ipykernel_launcher.py:59: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/imi432_006/anaconda3/envs/tf/lib/python3.5/site-packages/ipykernel_launcher.py:86: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/home/imi432_006/anaconda3/envs/tf/lib/python3.5/site-packages/ipykernel_launcher.py:90: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


...

epoch15 step123240 [D loss: 0.189994] [G loss: 0.039451]
epoch15 step123245 [D loss: -1.172355] [G loss: 0.187470]
epoch15 step123250 [D loss: -0.082334] [G loss: -0.144671]
epoch15 step123255 [D loss: -0.065174] [G loss: 0.267696]
epoch15 step123260 [D loss: -0.268208] [G loss: 0.255495]
epoch15 step123265 [D loss: -0.222192] [G loss: 0.088955]
epoch15 step123270 [D loss: -0.459188] [G loss: -0.091535]
epoch15 step123275 [D loss: 0.063228] [G loss: -0.206134]
epoch15 step123280 [D loss: 0.298901] [G loss: 0.024880]
epoch15 step123285 [D loss: -0.331807] [G loss: -0.626010]
epoch15 step123290 [D loss: -0.069211] [G loss: -0.152060]
epoch15 step123295 [D loss: -0.307984] [G loss: -0.141550]
epoch15 step123300 [D loss: -0.398400] [G loss: -0.158470]
epoch15 step123305 [D loss: 0.036907] [G loss: -0.133440]
epoch15 step123310 [D loss: -0.253080] [G loss: -0.324699]
epoch15 step123315 [D loss: -0.267065] [G loss: -0.035442]
epoch15 step123320 [D loss: 0.094367] [G loss: 0.247752]
epoch15 s

epoch15 step123910 [D loss: -0.004635] [G loss: -0.241044]
epoch15 step123915 [D loss: 0.216154] [G loss: -0.123488]
epoch15 step123920 [D loss: -0.164426] [G loss: -0.186828]
epoch15 step123925 [D loss: -0.412822] [G loss: -0.122214]
epoch15 step123930 [D loss: -0.393923] [G loss: -0.072410]
epoch15 step123935 [D loss: -0.123600] [G loss: -0.153341]
epoch15 step123940 [D loss: -0.123599] [G loss: -0.186645]
epoch15 step123945 [D loss: -0.422626] [G loss: -0.027026]
epoch15 step123950 [D loss: -0.410854] [G loss: -0.149157]
epoch15 step123955 [D loss: -0.197953] [G loss: -0.494426]
epoch15 step123960 [D loss: -0.526934] [G loss: -0.289112]
epoch15 step123965 [D loss: 0.035819] [G loss: -0.605492]
epoch15 step123970 [D loss: -0.567557] [G loss: -0.279011]
epoch15 step123975 [D loss: -0.055093] [G loss: -0.264203]
epoch15 step123980 [D loss: 0.209346] [G loss: -0.081027]
epoch15 step123985 [D loss: -0.081560] [G loss: -0.386282]
epoch15 step123990 [D loss: -0.230542] [G loss: -0.357919]


epoch15 step124580 [D loss: -0.426741] [G loss: -1.403721]
epoch15 step124585 [D loss: -0.118020] [G loss: -1.324353]
epoch15 step124590 [D loss: -0.359804] [G loss: -1.259273]
epoch15 step124595 [D loss: 0.327111] [G loss: -0.906910]
epoch15 step124600 [D loss: -0.811733] [G loss: -1.379143]
##############
[0.84747376 0.85336271 0.80190704 0.79903076 0.78400553 0.82744478
 0.88887625 0.83624397 0.80898587 0.84630527]
##########
epoch15 step124605 [D loss: -0.486644] [G loss: -1.294199]
epoch15 step124610 [D loss: -0.040051] [G loss: -1.012151]
epoch15 step124615 [D loss: -0.053589] [G loss: -0.983270]
epoch15 step124620 [D loss: -0.744837] [G loss: -1.123886]
epoch15 step124625 [D loss: -0.322371] [G loss: -1.568041]
epoch15 step124630 [D loss: 0.047884] [G loss: -0.943318]
epoch15 step124635 [D loss: 0.264374] [G loss: -1.195763]
epoch15 step124640 [D loss: -0.295261] [G loss: -1.056154]
epoch15 step124645 [D loss: 0.009959] [G loss: -0.792744]
epoch15 step124650 [D loss: -0.053604] 

epoch16 step125240 [D loss: 0.450253] [G loss: -0.688388]
epoch16 step125245 [D loss: -0.355226] [G loss: -0.436940]
epoch16 step125250 [D loss: -0.444889] [G loss: -0.661535]
epoch16 step125255 [D loss: 0.246237] [G loss: -0.840018]
epoch16 step125260 [D loss: 0.215911] [G loss: -0.827102]
epoch16 step125265 [D loss: 0.418460] [G loss: -0.536228]
epoch16 step125270 [D loss: -0.478774] [G loss: -0.471700]
epoch16 step125275 [D loss: 0.224706] [G loss: -0.461870]
epoch16 step125280 [D loss: -0.518752] [G loss: -0.060757]
epoch16 step125285 [D loss: -0.426897] [G loss: -0.488007]
epoch16 step125290 [D loss: -0.409134] [G loss: -0.130898]
epoch16 step125295 [D loss: -0.149257] [G loss: -0.455665]
epoch16 step125300 [D loss: -0.247724] [G loss: -0.079452]
epoch16 step125305 [D loss: -0.061388] [G loss: -0.298577]
epoch16 step125310 [D loss: -0.481485] [G loss: -0.083193]
epoch16 step125315 [D loss: -0.330246] [G loss: -0.232457]
epoch16 step125320 [D loss: -0.435861] [G loss: -0.093886]
ep

epoch16 step125910 [D loss: 0.108885] [G loss: 0.377858]
epoch16 step125915 [D loss: 0.050104] [G loss: 0.573739]
epoch16 step125920 [D loss: -0.062185] [G loss: 0.664720]
epoch16 step125925 [D loss: -0.366385] [G loss: 0.176365]
epoch16 step125930 [D loss: -0.168283] [G loss: 0.358753]
epoch16 step125935 [D loss: 0.022258] [G loss: 0.327729]
epoch16 step125940 [D loss: 0.015253] [G loss: 0.349473]
epoch16 step125945 [D loss: 0.102623] [G loss: 0.067385]
epoch16 step125950 [D loss: -0.085450] [G loss: 0.319821]
epoch16 step125955 [D loss: 0.047895] [G loss: 0.062881]
epoch16 step125960 [D loss: 0.086666] [G loss: 0.487121]
epoch16 step125965 [D loss: 0.684221] [G loss: -0.121010]
epoch16 step125970 [D loss: -0.468677] [G loss: 0.403270]
epoch16 step125975 [D loss: -0.134611] [G loss: 0.294280]
epoch16 step125980 [D loss: 0.233116] [G loss: 0.326389]
epoch16 step125985 [D loss: 0.277968] [G loss: 0.409901]
epoch16 step125990 [D loss: -0.364662] [G loss: 0.487749]
epoch16 step125995 [D l